In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn import *
import cv2
import time
import numpy as np
from lane_detection import detect_lane, detect_edges, region_of_interest, detect_line_segments, average_slope_intercept
from heading import *
import math
from sklearn.cluster import DBSCAN
%load_ext autoreload
%autoreload 3

In [ ]:
images_dir = Path('CarlaData/images_stress/').resolve()
files = list(images_dir.glob('*.png'))
files.sort()
files[:5]

In [ ]:
def load_frame(imname, cvt_color=True):
    img = cv2.imread(str(imname))
    if cvt_color:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
# show images
fig, ax = plt.subplots(1, len(files), figsize=(15, 5), dpi=300)
for i, f in enumerate(files):
    img = load_frame(f)
    ax[i].imshow(img)
    ax[i].axis('off')
    ax[i].set_title(f.name)
plt.show()

In [ ]:
from center_line import CenterLinePredictor

# roi_points=[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]],
#roi_points = [[0.41, 0.42], [0.58, 0.42], [0.58, 0.78], [0.41, 0.78]],
predictor = CenterLinePredictor(imsize=(640,360))

In [ ]:
for f in files:
    image = load_frame(f, cvt_color=False)
    plt.imshow(image)
    plt.show()
    resized_image = cv2.resize(
                image, predictor.imsize, interpolation=cv2.INTER_AREA)
    edges, mask = predictor.detect_edges(resized_image, return_mask=True)
    roi_mask = predictor.crop_to_roi(mask, invert=True)
    plt.imshow(roi_mask)
    plt.show()
    clustering_points = np.argwhere(roi_mask>0)
    db = DBSCAN(eps=5, min_samples=10).fit(clustering_points)
    labels = db.labels_
    # Number of clusters in labels, ignoring noise if present
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    print("Number of clusters is:", n_clusters_)
    # Now, you can iterate over your clusters and fit polynomials
    clusters = []
    for k in range(n_clusters_):
        class_member_mask = (labels == k)
        print("Number of points in cluster", k, "is:", np.sum(class_member_mask))
        cluster_points = clustering_points[class_member_mask]
        clusters.append([class_member_mask.sum(), cluster_points])
        plt.plot(cluster_points[:, 0], cluster_points[:, 1], 'o', label=f'cluster {k}')
        plt.legend()
    plt.show()
    
    # now merge clusters to get only 2 large ones and segment them to left and right
    thresh = image.shape[1] / 2
    clusters.sort(key=lambda x: x[0], reverse=True)
    clusters = clusters[:3] if len(clusters) > 3 else clusters
    left_cluster = None
    right_cluster = None
    mean_x_coord = lambda arr: np.mean(arr[:, 1])
    if len(clusters) > 2:
        first_cluster_size = clusters[0][0]
        second_cluster_size = clusters[1][0]
        third_cluster_size = clusters[2][0]
        if third_cluster_size >= 0.15 * min(first_cluster_size, second_cluster_size):
            mean_x_coord_first = mean_x_coord(clusters[0][1])
            mean_x_coord_second = mean_x_coord(clusters[1][1])
            mean_x_coord_third = mean_x_coord(clusters[2][1])
            dist_third_to_first = abs(mean_x_coord_first - mean_x_coord_third)
            dist_third_to_second = abs(mean_x_coord_second - mean_x_coord_third)
            if dist_third_to_first < dist_third_to_second:
                clusters[0][1] = np.concatenate((clusters[0][1], clusters[2][1]))
            else:
                clusters[1][1] = np.concatenate((clusters[1][1], clusters[2][1]))

    # now set left and right
    first_cluster_points = clusters[0][1]
    second_cluster_points = clusters[1][1]
    mean_x_coord_first = mean_x_coord(first_cluster_points)
    mean_x_coord_second = mean_x_coord(second_cluster_points)
    if mean_x_coord_first < mean_x_coord_second:
        left_cluster = first_cluster_points
        right_cluster = second_cluster_points
    else:
        left_cluster = second_cluster_points
        right_cluster = first_cluster_points
    plt.plot(left_cluster[:, 0], left_cluster[:, 1], 'o', label='left')
    plt.plot(right_cluster[:, 0], right_cluster[:, 1], 'o', label='right')
    plt.legend()
    plt.show()

In [ ]:
for i, f in enumerate(files):
    frame = load_frame(f, cvt_color=False)
    try:
        start = time.time()
        predictor.predict_to_poly(frame, visualize=True)
        end = time.time()
        print(f'Frame {i} took {(end-start)*1000} milliseconds')
    except Exception as e:
        print(e)